# Original MICE

In [ ]:
from __future__ import print_function
import os
import torch
import torch.backends.cudnn as cudnn
import argparse
import time
from torchvision import transforms, datasets
import torchvision
from dataset import *
from torch.utils.data import ConcatDataset
from util import *
from torch.utils import data

from ELBO import MiCE_ELBO
from sklearn import metrics





In [ ]:
dataset = 'stl10'
batch_size = 256
num_workers = 32

model = 'resnet34'
low_dim = 128
nu = 16384
tau = 1.0
gpu = 0

#model path
test_path = 'model_save/stl10_nu16384_resnet34_lr1.0_bsz256_epoch3000_tau1.0/ckpt_epoch_3000.pth'

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot

def plot_examples(images, labels):
    print(type(images[0]), type(labels))
    print(images.shape)
    w = 10
    h = 10
    fig = plt.figure(figsize=(10, 20))
    columns = 11
    rows = 12
    for i in range(10):
#         img = np.random.randint(10, size=(h,w))
        fig.add_subplot(rows, columns, i+1)
#         img = images[i] / 2 + 0.5   # unnormalize
        img = images[i]
        npimg = img.detach().cpu().numpy()   # convert from tensor
        
        plt.imshow(np.transpose(npimg, (1, 2, 0))) 
        plt.title('#{}: {}'.format(i, labels[i]))
    plt.show()
    

In [ ]:
import sys
def get_MiCE_performance(model, model_ema, elbo, dataloader, ndata, n_class, batchSize=100):
    model.eval()
    model_ema.eval()
    all_cluster_labels = []
    all_true_labels = []
    all_pi_labels = []

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            x1, x2, x3 = inputs[0].cuda(), inputs[1].cuda(), inputs[2].cuda()
            
            targets = targets.cuda()
            
            with torch.no_grad():
                f = model(x1)
                v = model_ema(x2)
                _, g = model(x3, True)

            loss, out, posterior, pi_logit = elbo(f, v, g)
#             print(posterior)
            feat_pseudo_label = torch.argmax(posterior, dim=1)  # batch x 1
            pi_label = torch.argmax(pi_logit, dim=1)
            all_cluster_labels.extend(list(feat_pseudo_label.data.cpu().numpy()))
            all_true_labels.extend(list(targets.data.cpu().numpy()))
            all_pi_labels.extend(list(pi_label.data.cpu().numpy()))

    all_true_labels = np.array(all_true_labels)
    all_cluster_labels = np.array(all_cluster_labels)
    all_pi_labels = np.array(all_pi_labels)

    print("True label stat:", np.bincount(all_true_labels.astype(int)))
    print("Cluster label stat:", np.bincount(all_cluster_labels.astype(int)))

    mice_acc = acc(all_true_labels, all_cluster_labels)
    pi_cluster_acc = acc(all_true_labels, all_pi_labels)

    nmi = metrics.normalized_mutual_info_score(labels_true=all_true_labels, labels_pred=all_cluster_labels)
    ari = metrics.adjusted_rand_score(labels_true=all_true_labels, labels_pred=all_cluster_labels)
    return mice_acc, pi_cluster_acc, nmi, ari

In [ ]:
image_size, mean, std, n_class = get_dataset_stat('stl10')
print(image_size, mean, std, n_class)
normalize = transforms.Normalize(mean=mean, std=std)

In [ ]:
    train_transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        normalize,
    ])

In [ ]:
train_dataset, test_dataset = create_dataset('stl10', train_transform, train_transform)
full_dataset = ConcatDataset([train_dataset, test_dataset])

n_full_data = len(full_dataset)
print(n_full_data)

In [ ]:
full_loader = torch.utils.data.DataLoader(full_dataset,
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=num_workers)

In [ ]:
model, model_ema = create_model(model, n_class)

In [ ]:
print(low_dim, n_full_data, nu, tau, n_class)
elbo = MiCE_ELBO(low_dim, n_full_data, nu, tau=tau, n_class=n_class).cuda(
        gpu)

In [ ]:
ckpt = torch.load(test_path)
model.load_state_dict(ckpt['model'])

model_ema.load_state_dict(ckpt['model_ema'])
elbo.load_state_dict(ckpt['elbo'])

print("==> loaded checkpoint '{}' (epoch {})".format(test_path, ckpt['epoch']))
print('==> done')

In [ ]:
model = model.cuda()
model_ema = model_ema.cuda()
model.eval()
model_ema.eval()
cudnn.benchmark = True

n_data = len(full_dataset)

In [ ]:
mice_acc, mice_pi_acc, mice_nmi, mice_ari = get_MiCE_performance(model, model_ema, elbo, full_loader, n_data, n_class)
print("CMoE average:", " NMI:", mice_nmi, "| Cluster ACC:", mice_acc, "| ARI:", mice_ari)

In [ ]:
# CMoE average:  NMI: 0.5568146990162488 | Cluster ACC: 0.6151538461538462 | ARI: 0.4589478341693674


## Attack training 

In [ ]:
import importlib
torch.cuda.empty_cache()

In [ ]:
import gan_attack 

In [ ]:
use_cuda=True
image_nc=3
epochs = 60
batch_size = 128
BOX_MIN = 0
BOX_MAX = 1
model_num_labels = 10
device = 'cuda'

In [ ]:
GAN = gan_attack.GAN_Attack(device, model, model_ema, elbo, model_num_labels, image_nc, BOX_MIN, BOX_MAX, 'new_stl10')

In [ ]:
# import sys

# # Holding the original output object. i.e. console out
# orig_stdout = sys.stdout

# # Opening the file to write file deletion logs.
# f = open('outgan_train256_s10.txt', 'w')

# # Changing standard out to file out. 
# sys.stdout = f
# # This will write to the file. 

# print("xyz") 

In [ ]:
GAN.train(full_loader, 300)

In [ ]:
# # Closing the file.
# f.close()



# # replacing the original output format to stdout.
# sys.stdout = orig_stdout

# # This will print onto the console.
# print("xyz") 

In [ ]:
ls models/

## Test

In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_new_stl10_epoch_300.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
import sys
def get_MiCE_adv_performance(model, model_ema, elbo, dataloader, ndata, n_class, batchSize=100):
    model.eval()
    model_ema.eval()
    all_cluster_labels = []
    all_true_labels = []
    all_pi_labels = []

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            x1, x2, x3 = inputs[0].cuda(), inputs[1].cuda(), inputs[2].cuda()
            
            perturbation = pretrained_G(x1)
            perturbation = torch.clamp(perturbation, -0.15, 0.15)
            adv_img1 = perturbation + x1
#             adv_img1 = torch.clamp(adv_img1, 0, 1)
            
            perturbation = pretrained_G(x2)
            perturbation = torch.clamp(perturbation, -0.15, 0.15)
            adv_img2 = perturbation + x2
#             adv_img2 = torch.clamp(adv_img2, 0, 1)
            
            perturbation = pretrained_G(x3)
            perturbation = torch.clamp(perturbation, -0.15, 0.15)
            adv_img3 = perturbation + x3
#             adv_img3 = torch.clamp(adv_img3, 0, 1)
            
            targets = targets.cuda()
            
            with torch.no_grad():
                f = model(adv_img1)
                v = model_ema(adv_img2)
                _, g = model(adv_img3, True)

            loss, out, posterior, pi_logit = elbo(f, v, g)
#             print(posterior)
            feat_pseudo_label = torch.argmax(posterior, dim=1)  # batch x 1
            pi_label = torch.argmax(pi_logit, dim=1)
            all_cluster_labels.extend(list(feat_pseudo_label.data.cpu().numpy()))
            all_true_labels.extend(list(targets.data.cpu().numpy()))
            all_pi_labels.extend(list(pi_label.data.cpu().numpy()))

    all_true_labels = np.array(all_true_labels)
    all_cluster_labels = np.array(all_cluster_labels)
    all_pi_labels = np.array(all_pi_labels)

    print("True label stat:", np.bincount(all_true_labels.astype(int)))
    print("Cluster label stat:", np.bincount(all_cluster_labels.astype(int)))

    mice_acc = acc(all_true_labels, all_cluster_labels)
    pi_cluster_acc = acc(all_true_labels, all_pi_labels)

    nmi = metrics.normalized_mutual_info_score(labels_true=all_true_labels, labels_pred=all_cluster_labels)
    ari = metrics.adjusted_rand_score(labels_true=all_true_labels, labels_pred=all_cluster_labels)
    return mice_acc, pi_cluster_acc, nmi, ari

In [ ]:
def save_examples(images, labels, noise=False, bno=0, adv=False, orig=False):
    print(type(images[0]), type(labels))
#     MEAN = torch.tensor([0.4914, 0.4822, 0.4465]).cuda() #c10
#     STD = torch.tensor([0.2023, 0.1994, 0.2010]).cuda() #c10
#     MEAN = torch.tensor([0.5071, 0.4867, 0.4408]).cuda()
#     STD = torch.tensor([0.2675, 0.2565, 0.2761]).cuda()
    MEAN = torch.tensor([0.5071, 0.4867, 0.4408]).cuda()
     
    STD = torch.tensor([0.2675, 0.2565, 0.2761]).cuda()
    
    for i in range(min(len(images), 20)):
        img = images[i]
        img = img * STD[:, None, None] + MEAN[:, None, None]
        npimg = img.detach().cpu().numpy()   # convert from tensor
        npimg = np.clip(npimg, 0, 1)
        if orig:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'../../Images/S10/MICE/orig/MICE_s10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
        if adv:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'../../Images/S10/MICE/adv/MICE_s10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
        if noise:
            npimg = npimg / 2 + 0.5 
            plt.imsave(f'../../Images/S10/MICE/noise/MICE_s10_b{bno}_{i}_noise_lab{labels[i]}.png', npimg.T, dpi=600)
            continue

In [ ]:
import sys
def get_MiCE_adv_performance_save(model, model_ema, elbo, dataloader, ndata, n_class, batchSize=100):
    model.eval()
    model_ema.eval()
    all_cluster_labels = []
    all_true_labels = []
    all_pi_labels = []

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            x1, x2, x3 = inputs[0].cuda(), inputs[1].cuda(), inputs[2].cuda()
            
            perturbation = pretrained_G(x1)
            perturbation = torch.clamp(perturbation, -0.15, 0.15)
            adv_img1 = perturbation + x1
#             adv_img1 = torch.clamp(adv_img1, 0, 1)
            
            perturbation = pretrained_G(x2)
            perturbation = torch.clamp(perturbation, -0.15, 0.15)
            adv_img2 = perturbation + x2
#             adv_img2 = torch.clamp(adv_img2, 0, 1)
            
            perturbation = pretrained_G(x3)
            perturbation = torch.clamp(perturbation, -0.15, 0.15)
            adv_img3 = perturbation + x3
#             adv_img3 = torch.clamp(adv_img3, 0, 1)
            
            targets = targets.cuda()
            
            with torch.no_grad():
                f = model(adv_img1)
                v = model_ema(adv_img2)
                _, g = model(adv_img3, True)

            loss, out, posterior, pi_logit = elbo(f, v, g)
#             print(posterior)
            feat_pseudo_label = torch.argmax(posterior, dim=1)  # batch x 1
            pi_label = torch.argmax(pi_logit, dim=1)
            all_cluster_labels.extend(list(feat_pseudo_label.data.cpu().numpy()))
            all_true_labels.extend(list(targets.data.cpu().numpy()))
            all_pi_labels.extend(list(pi_label.data.cpu().numpy()))
            save_examples(x3, targets.data.cpu().numpy(), bno=batch_idx, orig=True)
            save_examples(adv_img3, pi_label, bno=batch_idx, adv=True)
            save_examples(perturbation, pi_label, bno=batch_idx, noise=True)

    all_true_labels = np.array(all_true_labels)
    all_cluster_labels = np.array(all_cluster_labels)
    all_pi_labels = np.array(all_pi_labels)

    print("True label stat:", np.bincount(all_true_labels.astype(int)))
    print("Cluster label stat:", np.bincount(all_cluster_labels.astype(int)))

    mice_acc = acc(all_true_labels, all_cluster_labels)
    pi_cluster_acc = acc(all_true_labels, all_pi_labels)

    nmi = metrics.normalized_mutual_info_score(labels_true=all_true_labels, labels_pred=all_cluster_labels)
    ari = metrics.adjusted_rand_score(labels_true=all_true_labels, labels_pred=all_cluster_labels)
    return mice_acc, pi_cluster_acc, nmi, ari

In [ ]:
#saving samples

In [ ]:
mice_acc, mice_pi_acc, mice_nmi, mice_ari = get_MiCE_adv_performance_save(model, model_ema, elbo, full_loader, n_data, n_class)


In [ ]:
mice_acc, mice_pi_acc, mice_nmi, mice_ari = get_MiCE_adv_performance(model, model_ema, elbo, full_loader, n_data, n_class)
print("CMoE average:", " NMI:", mice_nmi, "| Cluster ACC:", mice_acc, "| ARI:", mice_ari)

# Transferability

In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../../Generator_Models/STL10/netG_CC_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

mice_acc, mice_pi_acc, mice_nmi, mice_ari = get_MiCE_adv_performance(model, model_ema, elbo, full_loader, n_data, n_class)
print("CMoE average:", " NMI:", mice_nmi, "| Cluster ACC:", mice_acc, "| ARI:", mice_ari)


In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../../Generator_Models/STL10/netG_NNM_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

mice_acc, mice_pi_acc, mice_nmi, mice_ari = get_MiCE_adv_performance(model, model_ema, elbo, full_loader, n_data, n_class)
print("CMoE average:", " NMI:", mice_nmi, "| Cluster ACC:", mice_acc, "| ARI:", mice_ari)


In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../../Generator_Models/STL10/netG_SCAN_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

mice_acc, mice_pi_acc, mice_nmi, mice_ari = get_MiCE_adv_performance(model, model_ema, elbo, full_loader, n_data, n_class)
print("CMoE average:", " NMI:", mice_nmi, "| Cluster ACC:", mice_acc, "| ARI:", mice_ari)


In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../../Generator_Models/STL10/netG_RUC_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

mice_acc, mice_pi_acc, mice_nmi, mice_ari = get_MiCE_adv_performance(model, model_ema, elbo, full_loader, n_data, n_class)
print("CMoE average:", " NMI:", mice_nmi, "| Cluster ACC:", mice_acc, "| ARI:", mice_ari)


In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../../Generator_Models/STL10/netG_SPICE_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

mice_acc, mice_pi_acc, mice_nmi, mice_ari = get_MiCE_adv_performance(model, model_ema, elbo, full_loader, n_data, n_class)
print("CMoE average:", " NMI:", mice_nmi, "| Cluster ACC:", mice_acc, "| ARI:", mice_ari)


# Additional Experiments

In [ ]:
import sys
def get_MiCE_adv_performance_norm(model, model_ema, elbo, dataloader, ndata, n_class, clamping=0.1):
    model.eval()
    model_ema.eval()
    all_cluster_labels = []
    all_true_labels = []
    all_pi_labels = []
    pertb_norm = 0.0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            p_n = 0.0
            x1, x2, x3 = inputs[0].cuda(), inputs[1].cuda(), inputs[2].cuda()
            
            perturbation = pretrained_G(x1)
            perturbation = torch.clamp(perturbation, -clamping, clamping)
            p_n += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()

            adv_img1 = perturbation + x1
#             adv_img1 = torch.clamp(adv_img1, 0, 1)
            pertb_norm += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()
            
            perturbation = pretrained_G(x2)
#             perturbation = torch.clamp(perturbation, -clamping, clamping)
            p_n += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()

            adv_img2 = perturbation + x2
#             adv_img2 = torch.clamp(adv_img2, 0, 1)
            
            perturbation = pretrained_G(x3)
            perturbation = torch.clamp(perturbation, -clamping, clamping)
            p_n += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()

            adv_img3 = perturbation + x3
#             adv_img3 = torch.clamp(adv_img3, 0, 1)
#             pertb_norm += perturbation
            
            targets = targets.cuda()
            
            with torch.no_grad():
                f = model(adv_img1)
                v = model_ema(adv_img2)
                _, g = model(adv_img3, True)

            loss, out, posterior, pi_logit = elbo(f, v, g)
#             print(posterior)
            feat_pseudo_label = torch.argmax(posterior, dim=1)  # batch x 1
            pi_label = torch.argmax(pi_logit, dim=1)
            all_cluster_labels.extend(list(feat_pseudo_label.data.cpu().numpy()))
            all_true_labels.extend(list(targets.data.cpu().numpy()))
            all_pi_labels.extend(list(pi_label.data.cpu().numpy()))

    all_true_labels = np.array(all_true_labels)
    all_cluster_labels = np.array(all_cluster_labels)
    all_pi_labels = np.array(all_pi_labels)

    print("True label stat:", np.bincount(all_true_labels.astype(int)))
    print("Cluster label stat:", np.bincount(all_cluster_labels.astype(int)))
    class_names = ['airplane',
                 'bird',
                 'car',
                 'cat',
                 'deer',
                 'dog',
                 'horse',
                 'monkey',
                 'ship',
                 'truck']
    labels_gt = torch.from_numpy(all_true_labels).cuda()
    labels_pred = torch.from_numpy(all_cluster_labels).cuda()
    
    clustering_stats_adv = eval_cus.check(labels_gt, labels_pred, 10, class_names, 
                                        compute_confusion_matrix=True, cf20=False, output_file2=f'MICE_s10_{clamping}_n{pertb_norm/len(dataloader)}.pdf')
    
    mice_acc = acc(all_true_labels, all_cluster_labels)
    pi_cluster_acc = acc(all_true_labels, all_pi_labels)

    nmi = metrics.normalized_mutual_info_score(labels_true=all_true_labels, labels_pred=all_cluster_labels)
    ari = metrics.adjusted_rand_score(labels_true=all_true_labels, labels_pred=all_cluster_labels)
    return mice_acc, pi_cluster_acc, nmi, ari, pertb_norm/len(dataloader)

In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128
device = 'cuda'
gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_new_stl10_epoch_300.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
import eval_cus

In [ ]:
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
# clamp = [0.0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
clamp = [0, 0.15, 1]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    acc2, mice_pi_acc, nmi, ari, norm = get_MiCE_adv_performance_norm(model, model_ema, elbo, full_loader, n_data, n_class, clamping=j)
    print(f'clamp {j} avg norm: {norm}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc2))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc2)

In [ ]:
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
# clamp = [0.0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
clamp = [0, , 1]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    acc2, mice_pi_acc, nmi, ari, norm = get_MiCE_adv_performance_norm(model, model_ema, elbo, full_loader, n_data, n_class, clamping=j)
    print(f'clamp {j} avg norm: {norm}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc2))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc2)

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)